<a href="https://colab.research.google.com/github/rodrigofer89/Machine-Learning/blob/main/Regress%C3%A3o_Linear_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Entendimento do problema**

É possível prever a altura dos filhos baseado nas alturas dos pais?

In [46]:
# Imports necessários
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [47]:
df = pd.read_csv('galton.csv')
df.head(5)

,family,father,mother,midparentHeight,children,childNum,gender,childHeight
0,1,78.5,67.0,75.43,4,1,male,73.2
1,1,78.5,67.0,75.43,4,2,female,69.2
2,1,78.5,67.0,75.43,4,3,female,69.0
3,1,78.5,67.0,75.43,4,4,female,69.0
4,2,75.5,66.5,73.66,4,1,male,73.5


In [48]:
# Quantos dados?
df.shape

(934, 8)

In [49]:
# Quais os tipos? Dados faltantes?
# não existem dados faltantes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   family           934 non-null    object 
 1   father           934 non-null    float64
 2   mother           934 non-null    float64
 3   midparentHeight  934 non-null    float64
 4   children         934 non-null    int64  
 5   childNum         934 non-null    int64  
 6   gender           934 non-null    object 
 7   childHeight      934 non-null    float64
dtypes: float64(4), int64(2), object(2)
memory usage: 58.5+ KB


In [50]:
# A coluna familia aparece como string pois existem valores alfabéticos, não será uma coluna útil
df['family'].values

array(['1', '1', '1', '1', '2', '2', '2', '2', '3', '3', '4', '4', '4',
       '4', '4', '5', '5', '5', '5', '5', '5', '6', '7', '7', '7', '7',
       '7', '7', '8', '8', '8', '9', '10', '11', '11', '11', '11', '11',
       '11', '11', '11', '12', '13', '13', '14', '14', '15', '15', '15',
       '16', '16', '16', '16', '16', '16', '16', '16', '16', '17', '17',
       '17', '17', '17', '17', '18', '18', '18', '19', '20', '20', '20',
       '20', '20', '20', '20', '20', '21', '21', '21', '22', '22', '22',
       '23', '23', '23', '23', '23', '23', '23', '24', '25', '25', '26',
       '26', '26', '26', '26', '27', '27', '27', '28', '28', '28', '28',
       '28', '28', '29', '29', '29', '30', '31', '31', '31', '31', '31',
       '31', '32', '32', '32', '32', '32', '33', '33', '33', '33', '33',
       '34', '35', '35', '35', '35', '35', '36', '36', '36', '36', '37',
       '37', '37', '37', '38', '38', '38', '38', '38', '38', '39', '39',
       '40', '40', '40', '40', '40', '41', '42', '42'

In [51]:
# Alguma info estatística útil?
df.describe()

,father,mother,midparentHeight,children,childNum,childHeight
count,934.000000,934.000000,934.000000,934.000000,934.000000,934.000000
mean,69.197109,64.089293,69.206773,6.171306,3.585653,66.745931
std,2.476479,2.290886,1.802370,2.729025,2.361410,3.579251
min,62.000000,58.000000,64.400000,1.000000,1.000000,56.000000
25%,68.000000,63.000000,68.140000,4.000000,2.000000,64.000000
50%,69.000000,64.000000,69.248000,6.000000,3.000000,66.500000
75%,71.000000,65.875000,70.140000,8.000000,5.000000,69.700000
max,78.500000,70.500000,75.430000,15.000000,15.000000,79.000000


In [52]:
# Quantos valores únicos por coluna?
df.nunique()

family             205
father              35
mother              29
midparentHeight    140
children            12
childNum            15
gender               2
childHeight         67
dtype: int64

**Limpeza dos dados**

In [53]:
# Removendo duplicatas, nesse caso não axistem dados duplicados
df.drop_duplicates(inplace=True)

In [54]:
# Removendo dados faltantes, nesse caso não existem dados faltantes
df.dropna(inplace=True)

In [55]:
df.shape

(934, 8)

Vamos trabalhar com dados em **m**?

In [56]:
def inch2m(inch):
  return inch*2.54/100

In [57]:
inch2m(78.5)

1.9939000000000002

In [58]:
# Aplicando a função para colocar os valores em metros
df['father'] = df['father'].apply(inch2m)

In [59]:
# Fazendo um for nas outras colunas de altura para metros
colunas = ['mother', 'midparentHeight', 'childHeight']
for coluna in colunas:
  df[coluna] = df[coluna].apply(inch2m)

Sabemos que os modelos só trabalham com números, então não podemos deixar a coluna GENDER como está! Precisamos transformá-la!

In [60]:
sexo = lambda x: 0 if x == 'male' else 1

In [61]:
# Criando uma função para transformar MALE E FEMALE em 0 e 1
def sex(x):
  if x == 'male':
    return 0
  else:
    return 1

In [62]:
# Aplicando a função sex na coluna GENDER
df['gender'] = df['gender'].apply(sex)

In [63]:
# Removendo colunas que não serão úteis
df2 = df.drop(columns=['family', 'childNum', 'children'])

**Modelagem**

In [64]:
# Separação de DADOS e LABEL => y = a*X + b
X = df2.drop(columns=['childHeight'])
y = df2['childHeight']

In [65]:
# Dividindo dados para TREINO e TESTE
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                    test_size = 0.3,
                                    random_state=43)

In [66]:
# Treinando o modelo
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [67]:
# Fazendo as predições
y_pred = modelo.predict(X_test)

**Avaliação das previsões**

In [68]:
# Erro quadrático médio
mean_squared_error(y_test, y_pred)

0.003095344698336929

In [69]:
# Erro absoluto médio
mean_absolute_error(y_test, y_pred)

0.04398745830307559

In [70]:
# R2_score
r2_score(y_test, y_pred)

0.6137640207302077

**Interpretando os coeficientes encontrados!**

Vamos dar uma olhada nos coeficientes encontrados para cada caracteristica:


In [71]:
modelo.intercept_

0.4841855410126339

In [72]:
modelo.coef_

array([-7.10165826e+12, -7.66979092e+12,  1.42033165e+13, -1.31347656e-01])

In [73]:
pd.DataFrame({
        'features': X_train.columns,
        'coeficientes': modelo.coef_
    }).sort_values(by='coeficientes', ascending=False)

,features,coeficientes
2,midparentHeight,1.420332e+13
3,gender,-1.313477e-01
0,father,-7.101658e+12
1,mother,-7.669791e+12


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 934 entries, 0 to 933
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   family           934 non-null    object 
 1   father           934 non-null    float64
 2   mother           934 non-null    float64
 3   midparentHeight  934 non-null    float64
 4   children         934 non-null    int64  
 5   childNum         934 non-null    int64  
 6   gender           934 non-null    int64  
 7   childHeight      934 non-null    float64
dtypes: float64(4), int64(3), object(1)
memory usage: 65.7+ KB


**Criando um modelo com duas variaveis, a Target e midparentHeight**

In [107]:
# Retirando as colunas que não seram usadas
df3 = df.drop(columns=['family', 'father', 'mother', 'children', 'childNum', 'gender'], axis=1)

In [108]:
# Separando a coluna Target
X = df3.drop(columns=['childHeight'])
y = df3['childHeight']

In [109]:
# Separando os dados em TREINO e TESTE
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=42)

In [110]:
# Criando o modelo 
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [111]:
# O valor predito é o X_test pois é o valor que não olhamos nunca, somente nas predições
y_pred = modelo.predict(X_test)

In [112]:
# Erro médio absoluto
mean_absolute_error(y_test, y_pred)

0.07417654868903377

In [113]:
# Erro médio quadrado
mean_squared_error(y_test, y_pred)

0.007662176577073954

In [114]:
# R2 score
r2_score(y_test, y_pred)

0.12716501104529288

In [115]:
modelo.intercept_

0.6462355974582854

In [116]:
modelo.coef_

array([0.59655342])

Nesse caso o modelo acabou ficando pior, acredito que algumas variaveis retiradas são importantes para o modelo

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 934 entries, 0 to 933
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   family           934 non-null    object 
 1   father           934 non-null    float64
 2   mother           934 non-null    float64
 3   midparentHeight  934 non-null    float64
 4   children         934 non-null    int64  
 5   childNum         934 non-null    int64  
 6   gender           934 non-null    int64  
 7   childHeight      934 non-null    float64
dtypes: float64(4), int64(3), object(1)
memory usage: 98.0+ KB


In [118]:
# Retirando as colunas que não seram usadas
df4 = df.drop(columns=['family', 'midparentHeight', 'children', 'childNum'])

In [120]:
# Separando a coluna Target do resto dos dados
X = df4.drop(columns=['childHeight'])
y = df4['childHeight']

In [122]:
# Separando o modelo em TREINO e TESTE
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=42)

In [124]:
# Criando o modelo
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [125]:
# O valor predito é o X_test pois é o valor que não olhamos nunca, somente nas predições 
y_pred = modelo.predict(X_test)

In [126]:
# Erro média absoluto
mean_absolute_error(y_test, y_pred)

0.043451097250684675

In [127]:
# Erro médio quadrado
mean_squared_error(y_test, y_pred)

0.003065611457941626

In [128]:
# R2 score
r2_score(y_test, y_pred)

0.6507816132770812

**Acretito que por ser um modelo muito simples não conseguimos melhoras significativas**

**FIM!!**